### Scenario 2: A cross-functional team with one data scientist working on an ML model
MLflow setup:

* tracking server: yes, local server
* backend store: sqlite database
* artifacts store: local filesystem
The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [9]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [10]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [11]:
mlflow.list_experiments()


[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.2, "random_state": 40}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: './mlruns/1/f5d755ed77c14ecbb0c67e136f54556a/artifacts'


In [18]:
mlflow.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

### Interacting with the model registry

In [19]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [20]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654240742100, description='', last_updated_timestamp=1654363055847, latest_versions=[<ModelVersion: creation_timestamp=1654363055847, current_stage='None', description='', last_updated_timestamp=1654363055847, name='iris-classifier', run_id='4ac1ea77ec434173b019d89edc333f92', run_link='', source='./mlruns/1/4ac1ea77ec434173b019d89edc333f92/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>], name='iris-classifier', tags={}>]

In [21]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2022/06/04 22:57:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 3
Created version '3' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1654363655142, current_stage='None', description='', last_updated_timestamp=1654363655142, name='iris-classifier', run_id='f5d755ed77c14ecbb0c67e136f54556a', run_link='', source='./mlruns/1/f5d755ed77c14ecbb0c67e136f54556a/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='3'>

### get the logged model from mlflow and use the model to make prediction

* get the last ran model uri
* load the corresponding model
* use the model to predict

In [22]:
exp = mlflow.get_experiment_by_name('my-experiment-1')
exp.experiment_id

'1'

In [23]:
df = mlflow.search_runs(exp.experiment_id)
df.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.C,params.random_state,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.log-model.history,tags.mlflow.user
0,f5d755ed77c14ecbb0c67e136f54556a,1,FINISHED,./mlruns/1/f5d755ed77c14ecbb0c67e136f54556a/ar...,2022-06-04 17:27:07.713000+00:00,2022-06-04 17:27:11.875000+00:00,0.966667,0.2,40,LOCAL,/usr/local/lib/python3.10/site-packages/ipyker...,"[{""run_id"": ""f5d755ed77c14ecbb0c67e136f54556a""...",bupadhy
1,4ac1ea77ec434173b019d89edc333f92,1,FINISHED,./mlruns/1/4ac1ea77ec434173b019d89edc333f92/ar...,2022-06-04 17:17:06.399000+00:00,2022-06-04 17:17:11.922000+00:00,0.960000,0.1,42,LOCAL,/usr/local/lib/python3.10/site-packages/ipyker...,"[{""run_id"": ""4ac1ea77ec434173b019d89edc333f92""...",bupadhy
2,59adff90f15a409bb28bfed3fc99da7c,1,FINISHED,./mlruns/1/59adff90f15a409bb28bfed3fc99da7c/ar...,2022-06-03 07:13:57.530000+00:00,2022-06-03 07:14:02.166000+00:00,0.960000,0.1,42,LOCAL,/usr/local/lib/python3.10/site-packages/ipyker...,"[{""run_id"": ""59adff90f15a409bb28bfed3fc99da7c""...",bupadhy


In [25]:
df.columns

Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.accuracy', 'params.C', 'params.random_state',
       'tags.mlflow.source.type', 'tags.mlflow.source.name',
       'tags.mlflow.log-model.history', 'tags.mlflow.user'],
      dtype='object')

In [24]:
# selecting the artificat uri of last run
last_run_uri = df.loc[df['end_time'].idxmax(), 'artifact_uri']
last_run_uri

'./mlruns/1/f5d755ed77c14ecbb0c67e136f54556a/artifacts'

In [35]:
## checking the params and metrics of last artifact uri
print('checking the params and metrics of last artifact uri')
(f"param C: {df[df.artifact_uri == last_run_uri].loc[0,'params.C']}",
f"param random state: {df[df.artifact_uri == last_run_uri].loc[0,'params.random_state']}",
f"metrics  accuracy: {df[df.artifact_uri == last_run_uri].loc[0,'metrics.accuracy']}")


checking the params and metrics of last artifact uri


('param C: 0.2',
 'param random state: 40',
 'metrics  accuracy: 0.9666666666666667')

In [36]:
# loading the registered model from mlflow
model = mlflow.pyfunc.load_model(f'{last_run_uri}/models/')
print(model)

mlflow.pyfunc.loaded_model:
  artifact_path: models
  flavor: mlflow.sklearn
  run_id: f5d755ed77c14ecbb0c67e136f54556a



In [37]:
# prediction
X, y = load_iris(return_X_y=True)
y_pred = model.predict(X)
accuracy_score(y, y_pred)

0.9666666666666667